In [1]:
from datasets.dataset import AzureDataset
import config
path_to_bound = 'configs/Azure/apartment.yaml'
cfg = config.load_config(path_to_bound)
dataset = AzureDataset(cfg=cfg, basedir=cfg['data']['datadir'])



In [17]:
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm
H, W = dataset.H, dataset.W
fx, fy, cx, cy = dataset.fx, dataset.fy, dataset.cx, dataset.cy
K = o3d.camera.PinholeCameraIntrinsic(W, H, fx, fy, cx, cy)
voxel_length = 0.02
volume = o3d.pipelines.integration.ScalableTSDFVolume(voxel_length=voxel_length, sdf_trunc=0.04,
                                            color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

for i, frame in enumerate(tqdm(dataset)):
    if i % 10 != 0:
        continue
    rgb, depth = frame["rgb"].cpu().numpy(), frame["depth"].cpu().numpy()
    c2w = frame["c2w"].cpu().numpy()
    rgb = rgb * 255
    rgb = rgb.astype(np.uint8)
    rgb = o3d.geometry.Image(rgb)
    depth = depth.astype(np.float32)
    depth = o3d.geometry.Image(depth)
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb, depth, depth_scale=1.0,
                                                                depth_trunc=8.0,
                                                                convert_rgb_to_intensity=False)
    c2w[:3, 1] *= -1
    c2w[:3, 2] *= -1
    w2c = np.linalg.inv(c2w)
    # requires w2c
    volume.integrate(rgbd, K, w2c)

print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()

100%|██████████| 12595/12595 [09:40<00:00, 21.68it/s]


Extract a triangle mesh from the volume and visualize it.


TriangleMesh with 429052 points and 725198 triangles.

In [23]:
from tools.vis_cameras import visualize, draw_cuboid
#mesh_gt = o3d.io.read_triangle_mesh("output\Apartment\demo\mesh_track12594.ply")
mesh_gt = o3d.io.read_triangle_mesh("output\Apartment\demo\mesh_track500.ply")
cfg = config.load_config('configs/IPhone/statue.yaml')
scene_bound = np.array(cfg["mapping"]["bound"])
mcube_bound = np.array(cfg["mapping"]["marching_cubes_bound"])

cube_large = draw_cuboid(scene_bound)
cube_small = draw_cuboid(mcube_bound)

visualize(extrinsics=None, things_to_draw=[cube_large, cube_small, mesh_gt])
#visualize(extrinsics=None, things_to_draw=[cube_large, cube_small, mesh])